In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import talib as ta
import seaborn as sns
import fix_yahoo_finance

from pandas_datareader import data as web
from sklearn import mixture as mix
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
df = web.get_data_yahoo('SPY', start = '2000-01-01', end = '2017-08-08')
df = df[['Open', 'High', 'Low', 'Close']]

In [3]:
df.head()

,Open,High,Low,Close
Date,,,,
1999-12-31,146.843704,147.500000,146.250000,146.8750
2000-01-03,148.250000,148.250000,143.875000,145.4375
2000-01-04,143.531204,144.062500,139.640594,139.7500
2000-01-05,139.937500,141.531204,137.250000,140.0000
2000-01-06,139.625000,141.500000,137.750000,137.7500


In [4]:
n = 10
t = 0.8

In [5]:
split = int(t * len(df))

In [7]:
split

3543

In [8]:
df['high'] = df['High'].shift(1)
df['Low'] = df['Low'].shift(1)
df['Close'] = df['Close'].shift(1)

In [11]:
df['RSI'] = ta.RSI(np.array(df['Close']), timeperiod = n) 
df['SMA'] = df['Close'].rolling(window = n).mean()
df['Corr'] = df['SMA'].rolling(window = n).corr(df['Close'])
df['SAR'] = ta.SAR(np.array(df['High']), np.array(df['Low']), 0.2,0.2)
df['ADX'] = ta.ADX(np.array(df['High']), np.array(df['Low']),
                    np.array(df['Close']), timeperiod = n)

In [12]:
df['Return'] = np.log(df['Open'] / df['Open'].shift(1))

In [14]:
df.head(20)

,Open,High,Low,Close,high,RSI,SMA,Corr,SAR,ADX,Return
Date,,,,,,,,,,,
1999-12-31,146.843704,147.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,148.250000,148.250000,146.250000,146.875000,147.500000,NaN,NaN,NaN,NaN,NaN,0.009531
2000-01-04,143.531204,144.062500,143.875000,145.437500,148.250000,NaN,NaN,NaN,148.250000,NaN,-0.032348
2000-01-05,139.937500,141.531204,139.640594,139.750000,144.062500,NaN,NaN,NaN,147.375000,NaN,-0.025357
2000-01-06,139.625000,141.500000,137.250000,140.000000,141.531204,NaN,NaN,NaN,145.828119,NaN,-0.002236
2000-01-07,140.312500,145.750000,137.750000,137.750000,141.500000,NaN,NaN,NaN,137.250000,NaN,0.004912
2000-01-10,146.250000,146.906204,140.062500,145.750000,145.750000,NaN,NaN,NaN,137.250000,NaN,0.041445
2000-01-11,145.812500,146.093704,145.031204,146.250000,146.906204,NaN,NaN,NaN,137.750000,NaN,-0.002996
2000-01-12,144.593704,144.593704,143.500000,144.500000,146.093704,NaN,NaN,NaN,139.581241,NaN,-0.008394


In [15]:
df = df.dropna()

In [17]:
df.head(20)

,Open,High,Low,Close,high,RSI,SMA,Corr,SAR,ADX,Return
Date,,,,,,,,,,,
2000-01-31,135.812500,139.671799,135.531204,135.875000,140.062500,30.348514,142.818741,0.290914,142.328977,31.734838,-0.026341
2000-02-01,139.750000,141.687500,135.000000,139.562500,139.671799,41.344141,142.078120,0.513762,135.000000,32.148366,0.028580
2000-02-02,141.281204,142.250000,138.531204,140.937500,141.687500,44.945065,141.590620,0.543169,135.000000,32.024322,0.010897
2000-02-03,140.875000,143.250000,140.375000,141.062500,142.250000,45.284361,140.996870,0.477415,135.000000,31.031837,-0.002879
2000-02-04,143.187500,144.000000,140.000000,143.187500,143.250000,50.989641,140.840620,0.151582,136.650000,29.490307,0.016282
2000-02-07,142.562500,142.781204,142.125000,142.593704,144.000000,49.390651,140.656241,-0.291728,138.120000,28.102930,-0.004374
2000-02-08,143.968704,144.562500,141.437500,142.375000,142.781204,48.764832,140.859370,-0.362776,139.296000,25.296789,0.009815
2000-02-09,144.468704,144.468704,143.625000,144.312500,144.562500,54.446400,141.096870,-0.577724,140.349300,22.771262,0.003467
2000-02-10,141.625000,142.562500,141.265594,141.281204,144.468704,45.646944,141.143741,-0.688139,141.191940,22.302749,-0.019880


In [22]:
ss = StandardScaler()
df.shape

(4409, 11)

In [23]:
x = np.reshape(ss.fit_transform(df[:split]), (-1, df.shape[1]))
x.shape

(3543, 11)

In [ ]:
regime = unsup.predict(np.reshape(ss.fit_transform(df[split:]),(-1, df.shape[1])))